In [3]:
#libraries
import numpy as np

## README

Here I'm just doing a bit of wrangling to make different codeblocks that I need for BEAST xml files. At some point this can start to the form the basis of a broader program to fully automate xml file generation without using BEAUti. For now it just generates some chunks that BEAUti won't make right now. 

In [4]:
#make generalDataType code block
with open('/Users/alliblk/Desktop/gitrepos/zika-usvi/data/glm/indexed-countries-45.tsv','rU') as infile:
    with open('/Users/alliblk/Desktop/gitrepos/zika-usvi/beast/generalDataType_countryList.txt', 'w') as outfile:
        for line in infile:
            if line.startswith('country'):
                continue
            else:
                outfile.write('\t\t')
                outfile.write('<state code="{}"/>'.format(line.split('\t')[0]))
                outfile.write('\n')

In [35]:
#make markov rewards code block

#link countries to an index to ensure proper ordering is preserved
#current file ordering is correct
index_dict = {}
counter = 0
with open('/Users/alliblk/Desktop/gitrepos/zika-usvi/data/glm/indexed-countries-45.tsv','rU') as infile:
    for line in infile:
        if line.startswith('country'):
            continue
        else:
            index_dict[counter] = line.split('\t')[0] #use country, not region
            counter +=1
            
#initialize empty markov rewards matrix
#need the same number of rows and columns as you have demes. I.E. here it needs to be 45 by 45.
reward_matrix = np.zeros((len(index_dict),len(index_dict)))
for i in range(len(index_dict)): #for row in matrix
    for j in range(len(index_dict)): #for column in matrix
        if i == j: #if indexes match, i.e. the countries are the same:
            reward_matrix[i,j] += 1 #that markov reward entry should == 1.
        else:
            continue #all other entries should be 0.
print reward_matrix

reward_xml_code ='''\t\t\t<parameter id="{country}_reward" value="{values}"/>\n'''

with open('/Users/alliblk/Desktop/gitrepos/zika-usvi/beast/markov_rewards_block.txt', 'w') as outfile:
    outfile.write('\t\t<rewards>\n')
    for i in range(len(reward_matrix)):
        outfile.write(reward_xml_code.format(country=index_dict[i], values = ' '.join(str(value) for value in reward_matrix[i])))
    outfile.write('\t\t</rewards>\n')



[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]]


In [52]:
#initialize empty markov rewards matrix
#need the same number of rows and columns as you have demes. I.E. here it needs to be 45 by 45.
count_matrix = np.zeros((len(index_dict),len(index_dict)))
for i in range(len(index_dict)): #for row in matrix
    for j in range(len(index_dict)): #for column in matrix
        if i != j: #if indexes match, i.e. the countries are the same:
            count_matrix[i,j] += 1 #that markov reward entry should == 1.
        else:
            continue #diagonal entries should be 0.
values = []
for i in range(len(count_matrix)):
    values.append(' '.join(str(value) for value in count_matrix[i]))
string_values =  ' '.join(values)
#print count_matrix


count_xml_code = '''\t\t<parameter id="country.count" value="{values}"/>\n'''
with open('/Users/alliblk/Desktop/gitrepos/zika-usvi/beast/markov_countryCounts_block.txt', 'w') as outfile:
    outfile.write(count_xml_code.format(values = string_values))

Below, I'm making the Markov Jump matrices. Note that these are formatted differently from how the GLM predictor matrices are formatted. Notably both the size of the matrices are different (Markov jumps matrix DO CONTAIN the diagonal values while the GLM predictor matrices do not), and the indexing is different (see figure below). 

**GLM Predictor indexing**

XXXXXXX| usa |canada| mexico
-------|-----|-------|-------
usa    |XXXXX|  0    | 1
canada |3    |XXXXXXX| 2
mexico |4    |  5    | XXXXXX

**Markov jump matrix indexing**

XXXXXXX| usa |canada| mexico
-------|-----|-------|-------
usa    |0    |  1    | 2
canada |3    |  4    | 5
mexico |6    |  7    | 8

The markov jump matrices are made for both `to_<country>` and `from_<country>` for all countries that you want Markov jumps for. The matrices contain values of 1 for all instances where you are going to/from the indicated countries. All other values in the matrix, including diagonals (to and from country are the same country), get a value of 0.

To make sure that the code is working properly I've 'remade' the jump matrices for `AC1_to` and `AC1_from` from Lemey et al (2014) paper Unifying Viral Genetics and Human Transportation Data to Predict the Global Transmission Dynamics of Human Influenza H3N2. The xml is available as part of the supplement online. Here I'm making the matries and then seeing if I can find them in the xml, ensuring they match what I think they should.


In [40]:
counter2 = 0
lemey_states = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
lemey_dict = {}
for from_state in lemey_states: #from_states represent rows in the matrix
    for to_state in lemey_states: #to_states represent columns in the matrix
        lemey_dict[counter2] = (from_state,to_state)
        counter2 +=1

ac1_to = []
for key in lemey_dict.keys():
    if lemey_dict[key][1] == 1 and lemey_dict[key][0] != lemey_dict[key][1]: #if second value in tuple, i.e. the 'to' country, is canada
        ac1_to.append(1)
    else:
        ac1_to.append(0)

ac1_to_xml = ' '.join(str(value) for value in ac1_to)
print ac1_to_xml

ac1_from = []
for key in lemey_dict.keys():
    if lemey_dict[key][0] == 1 and lemey_dict[key][0] != lemey_dict[key][1]: #if second value in tuple, i.e. the 'to' country, is canada
        ac1_from.append(1)
    else:
        ac1_from.append(0)

ac1_from_xml = ' '.join(str(value) for value in ac1_from)
print ac1_from_xml

0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


In [44]:
# Lemey et al predictors look good. Moving on to my predictors now
with open('/Users/alliblk/Desktop/gitrepos/zika-usvi/data/glm/indexed-countries-45.tsv') as file:
    countries_list = [line.strip().split('\t')[0] for line in file if not line.startswith('country')]
    
print countries_list #this should be ordered north to south
print '\n {} countries included in analysis'.format(len(countries_list)) #this should be 45

countries_dict = {}
count_ints = 0
for from_state in countries_list:
    for to_state in countries_list:
        countries_dict[count_ints] = (from_state,to_state) #tuple[0] == from_state, tuple[1] == to_state
        count_ints +=1

print len(countries_dict)
assert len(countries_dict) == len(countries_list)**2, 'markov jump matrix wrong size, check that diagonals are being included'

to_usvi_list = []
from_usvi_list = []

#from usvi list, from_states is tuple[0]
for key in countries_dict.keys():
    if countries_dict[key][0] == 'united_states_virgin_islands' and countries_dict[key][0] != countries_dict[key][1]: #from country is usvi, to country is different
        from_usvi_list.append(1)
    else:
        from_usvi_list.append(0)

#to usvi list, to_states is tuple[1]
for key in countries_dict.keys():
    if countries_dict[key][1] == 'united_states_virgin_islands' and countries_dict[key][0] != countries_dict[key][1]: #to country is usvi, from country is different
        to_usvi_list.append(1)
    else:
        to_usvi_list.append(0)

print 45**2
print len(to_usvi_list)
print len(from_usvi_list)

['canada', 'united_states', 'bermuda', 'mexico', 'belize', 'guatemala', 'honduras', 'el_salvador', 'nicaragua', 'costa_rica', 'panama', 'bahamas', 'cuba', 'turks_caicos_islands', 'cayman_islands', 'jamaica', 'haiti', 'dominican_republic', 'puerto_rico', 'united_states_virgin_islands', 'saint_kitts_nevis', 'antigua_barbuda', 'guadeloupe', 'dominica', 'martinique', 'saint_lucia', 'saint_vincent_grenadines', 'barbados', 'grenada', 'trinidad_tobago', 'curacao', 'aruba', 'french_guiana', 'suriname', 'guyana', 'venezuela', 'colombia', 'ecuador', 'peru', 'bolivia', 'brazil', 'paraguay', 'uruguay', 'argentina', 'chile']

 45 countries included in analysis
2025
2025
2025
2025


In [42]:
xml_jump_code ='''\t\t<parameter id="{}_usvi" value="{values}"/>\n'''
with open('/Users/alliblk/Desktop/gitrepos/zika-usvi/beast/markov_jumps_block.txt', 'w') as outfile:
    outfile.write(xml_jump_code.format('to', values = ' '.join(str(value) for value in to_usvi_list)))
    outfile.write(xml_jump_code.format('from', values = ' '.join(str(value) for value in from_usvi_list)))